In [ ]:
import pandas as pd


In [ ]:
churn=pd.read_csv("Churn_Modelling.csv")

In [ ]:
churn.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
x=churn.iloc[:,3:13]
y=churn.iloc[:,13]

In [ ]:
##create dummies

gender=pd.get_dummies(x['Gender'],drop_first=True)

In [ ]:
x=pd.concat([x,gender],axis=1)

In [ ]:
x=x.drop(['Geography','Gender'],axis=1)

In [ ]:
x.head(5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male
0,619,42,2,0.00,1,1,1,101348.88,0
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,0
3,699,39,1,0.00,2,0,0,93826.63,0
4,850,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [ ]:
x_train=pd.DataFrame(x_train)
x_train.head(5)

,0,1,2,3,4,5,6,7,8
0,-0.735507,0.015266,0.008860,0.673160,2.535034,-1.553624,-1.034460,-1.640810,-1.087261
1,1.024427,-0.652609,0.008860,-1.207724,0.804242,0.643657,-1.034460,-0.079272,-1.087261
2,0.808295,-0.461788,1.393293,-0.356937,0.804242,0.643657,0.966688,-0.996840,-1.087261
3,0.396614,-0.080145,0.008860,-0.009356,-0.926551,0.643657,0.966688,-1.591746,0.919743
4,-0.467915,1.255605,0.701077,-1.207724,0.804242,0.643657,0.966688,1.283302,0.919743


In [ ]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout
from kerastuner.tuners import RandomSearch
from tensorflow.keras import layers

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 7.5MB/s 


In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=128),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid',))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir1',
    project_name='churn')



In [ ]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Trial 10 Complete [00h 00m 10s]
val_accuracy: 0.8547999858856201

Best val_accuracy So Far: 0.8628000020980835
Total elapsed time: 00h 01m 38s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir1/churn
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 2
units_0: 288
units_1: 32
learning_rate: 0.01
units_2: 288
units_3: 416
units_4: 416
units_5: 288
units_6: 288
units_7: 416
units_8: 288
units_9: 160
units_10: 288
units_11: 160
units_12: 32
units_13: 416
units_14: 416
units_15: 416
units_16: 160
Score: 0.8628000020980835
Trial summary
Hyperparameters:
num_layers: 11
units_0: 416
units_1: 32
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
Score: 0.8611999750137329
Trial summary
Hyperparameters:
num_layers: 7
units_0: 416
units_1: 160
learning_rate: 0.001
units_2: 160
units_3: 288
units_4: 160
units_5: 416
units_6: 288
units_7: 32
units_8: 416
units_9: 160
units_10: 288
units_11: 416
units_12: 288
units_13: 416
units_14: 416
units_15: 416
units_16: 288
Score: 0.8604000210762024
Trial summary
Hyperparamet

In [ ]:
classsifier=Sequential()

In [ ]:
x_train.shape

(7500, 9)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(9,)))
model.add(tf.keras.layers.Dense(288,activation='relu',kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dense(32,activation='relu',kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid',kernel_initializer='glorot_normal'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=30)

Epoch 1/30
235/235 [==============================] - 1s 2ms/step - loss: 0.4899 - accuracy: 0.7865
Epoch 2/30
235/235 [==============================] - 0s 2ms/step - loss: 0.3876 - accuracy: 0.8310
Epoch 3/30
235/235 [==============================] - 0s 2ms/step - loss: 0.3653 - accuracy: 0.8502
Epoch 4/30
235/235 [==============================] - 1s 2ms/step - loss: 0.3446 - accuracy: 0.8599
Epoch 5/30
235/235 [==============================] - 0s 2ms/step - loss: 0.3564 - accuracy: 0.8530
Epoch 6/30
235/235 [==============================] - 0s 2ms/step - loss: 0.3222 - accuracy: 0.8666
Epoch 7/30
235/235 [==============================] - 0s 2ms/step - loss: 0.3260 - accuracy: 0.8636
Epoch 8/30
235/235 [==============================] - 0s 2ms/step - loss: 0.3283 - accuracy: 0.8653
Epoch 9/30
235/235 [==============================] - 1s 2ms/step - loss: 0.3251 - accuracy: 0.8666
Epoch 10/30
235/235 [==============================] - 1s 2ms/step - loss: 0.3273 - accuracy: 0.8629

In [ ]:
model.evaluate(x_test,y_test)

79/79 [==============================] - 0s 2ms/step - loss: 0.3720 - accuracy: 0.8596


[0.3720104992389679, 0.8596000075340271]

**Here in the churn model we need to reduce FN as much as possible, so we will look not only accuracy but also FN **

In [ ]:
y_predict=model.predict(x_test)
y_predict=(y_predict>0.25)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_predict)
acc=accuracy_score(y_test,y_predict)

In [ ]:
cm

array([[1721,  270],
       [ 191,  318]])

In [ ]:
acc

0.8156